In [1]:
sc

In [5]:
!rm -rf metastore_db/*.lck

from pyspark.sql import SQLContext
sqlc = SQLContext(sc)

## Using User Defined Functions

In [6]:
from numpy import NaN
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import DoubleType

udf1 = UserDefinedFunction(lambda x: NaN if x > 0.5 else x, DoubleType())
udf2 = UserDefinedFunction(lambda x: NaN if x > 1.0 else x, DoubleType())

In [7]:
from pyspark.sql.functions import rand, randn

df = sqlc.range(0, 5).withColumn("uniform", rand(5)).withColumn("normal", randn(5))

dfnan = df.withColumn("nanUniform", udf1("uniform")) \
          .withColumn("nanNormal", udf2("normal")) \
          .drop("uniform").withColumnRenamed("nanUniform", "uniform") \
          .drop("normal").withColumnRenamed("nanNormal", "normal")

In [8]:
dfnan.show()

+---+-------------------+--------------------+
| id|            uniform|              normal|
+---+-------------------+--------------------+
|  0|0.47611851579756026|-0.21311682946326227|
|  1|0.06498948189958098|-0.05248092572410684|
|  2|                NaN|                 NaN|
|  3| 0.1982919638208397|  -0.256535324205377|
|  4|0.12030715258495939|  -0.506853671746243|
+---+-------------------+--------------------+



## NA Functions

### Drop

In [9]:
dfnan.na.drop(how='all',subset=['uniform','normal']).show()

+---+-------------------+--------------------+
| id|            uniform|              normal|
+---+-------------------+--------------------+
|  0|0.47611851579756026|-0.21311682946326227|
|  1|0.06498948189958098|-0.05248092572410684|
|  3| 0.1982919638208397|  -0.256535324205377|
|  4|0.12030715258495939|  -0.506853671746243|
+---+-------------------+--------------------+



### Replace

In [10]:
dfnan.na.replace([NaN],[0.0], 'uniform').show()

+---+-------------------+--------------------+
| id|            uniform|              normal|
+---+-------------------+--------------------+
|  0|0.47611851579756026|-0.21311682946326227|
|  1|0.06498948189958098|-0.05248092572410684|
|  2|                0.0|                 NaN|
|  3| 0.1982919638208397|  -0.256535324205377|
|  4|0.12030715258495939|  -0.506853671746243|
+---+-------------------+--------------------+



### Fill

In [11]:
dfnan.na.fill({'uniform': 0.0, 'normal': 1.0}).show()

+---+-------------------+--------------------+
| id|            uniform|              normal|
+---+-------------------+--------------------+
|  0|0.47611851579756026|-0.21311682946326227|
|  1|0.06498948189958098|-0.05248092572410684|
|  2|                0.0|                 1.0|
|  3| 0.1982919638208397|  -0.256535324205377|
|  4|0.12030715258495939|  -0.506853671746243|
+---+-------------------+--------------------+



In [50]:
dfsummary = dfnan.drop("id").na.drop(how='all').describe()
dfsummary.show()

+-------+-------------------+--------------------+
|summary|            uniform|              normal|
+-------+-------------------+--------------------+
|  count|                  4|                   4|
|   mean|0.21492677852573508|-0.25724668778474724|
| stddev|0.18251201532325845| 0.18813170403403176|
|    min|0.06498948189958098|  -0.506853671746243|
|    max|0.47611851579756026|-0.05248092572410684|
+-------+-------------------+--------------------+



In [54]:
uniformMean = float(dfsummary.rdd.collect()[1][1])
print uniformMean

0.214926778526


In [52]:
pandasSummary = dfsummary.toPandas().set_index("summary")
pandasSummary

,uniform,normal
summary,,
count,4,4
mean,0.21492677852573508,-0.25724668778474724
stddev,0.18251201532325845,0.18813170403403176
min,0.06498948189958098,-0.506853671746243
max,0.47611851579756026,-0.05248092572410684


In [55]:
uniformMean = float(pandasSummary.loc['mean'].uniform)
print uniformMean

0.214926778526


In [56]:
normalMean = float(pandasSummary.loc['mean'].normal)
print normalMean

-0.257246687785


In [57]:
dfnan.na.fill({"uniform": uniformMean, "normal": normalMean}).show()

+---+-------------------+--------------------+
| id|            uniform|              normal|
+---+-------------------+--------------------+
|  0|0.47611851579756026|-0.21311682946326227|
|  1|0.06498948189958098|-0.05248092572410684|
|  2|     0.214926778526|     -0.257246687785|
|  3| 0.1982919638208397|  -0.256535324205377|
|  4|0.12030715258495939|  -0.506853671746243|
+---+-------------------+--------------------+



## Duplicates

In [62]:
dfDuplicates = dfnan.union(sc.parallelize([(5,1,1),(6,1,1)]).toDF())

In [63]:
dfDuplicates.show()

+---+-------------------+--------------------+
| id|            uniform|              normal|
+---+-------------------+--------------------+
|  0|0.47611851579756026|-0.21311682946326227|
|  1|0.06498948189958098|-0.05248092572410684|
|  2|                NaN|                 NaN|
|  3| 0.1982919638208397|  -0.256535324205377|
|  4|0.12030715258495939|  -0.506853671746243|
|  5|                1.0|                 1.0|
|  6|                1.0|                 1.0|
+---+-------------------+--------------------+



In [64]:
dfDuplicates.dropDuplicates(["uniform","normal"]).show()

+---+-------------------+--------------------+
| id|            uniform|              normal|
+---+-------------------+--------------------+
|  1|0.06498948189958098|-0.05248092572410684|
|  2|                NaN|                 NaN|
|  5|                1.0|                 1.0|
|  4|0.12030715258495939|  -0.506853671746243|
|  0|0.47611851579756026|-0.21311682946326227|
|  3| 0.1982919638208397|  -0.256535324205377|
+---+-------------------+--------------------+

